In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import random as rm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as img

%matplotlib inline

#from sklearn.tree import DecisionTreeClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC,LinearSVC
#from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from sklearn.preprocessing import Imputer,Normalizer,scale,StandardScaler ,MinMaxScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV , KFold, cross_val_score
from sklearn.metrics import confusion_matrix,mean_squared_log_error,mean_squared_error,mean_absolute_error,r2_score,accuracy_score,precision_score, recall_score,f1_score



from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
df = train.copy()
df_test = test.copy()
print(train.shape,test.shape)

In [ ]:
print(df.shape)
print(df_test.shape)

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df_test.info()

In [ ]:
df.isnull().any().sum()

In [ ]:
df_test.isnull().any().sum()

In [ ]:
#visulaising the dataset

img = np.random.randint(0,42000,size=4)
print(img)

In [ ]:
fig,axi = plt.subplots(2,2, figsize=(10,10))
axi[0,0].imshow(df.drop('label',axis=1).values[img[0]].reshape(28,28),cmap='gray')
axi[0,1].imshow(df.drop('label',axis=1).values[img[1]].reshape(28,28),cmap='gray')
axi[1,0].imshow(df.drop('label',axis=1).values[img[2]].reshape(28,28),cmap='gray')
axi[1,1].imshow(df.drop('label',axis=1).values[img[3]].reshape(28,28),cmap='gray')

In [ ]:
df['label'].value_counts()

In [ ]:
#data preprocessing

#X_train=train.iloc[:,1:]
#y_train=train.iloc[:,0]
#X_test=test.iloc[:,0:]
y_train=train['label']
X_train=train.drop(labels=['label'],axis=1)
X_test=test
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

In [ ]:
sns.countplot(y_train)
y_train.value_counts()

In [ ]:
#label encoding
y_train=to_categorical(y_train,num_classes=10)
#y_test=to_categorial(y_test,num_classes=10)
print(y_train.shape)
#print(y_test.shape)

In [ ]:
X_train = X_train.astype("float32")/255.0
X_test = X_test.astype("float32")/255.0

In [ ]:
X_train= X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
#df_test = df_test.reshape(df_test.shape[0],28,28,1)
print(X_train.shape,X_test.shape)

In [ ]:

#df_test = df_test.astype("float32")/255

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=66)

In [ ]:
print(X_train.shape,X_val.shape,y_train.shape,y_val.shape)

**Normalization, Reshape and Label Encoding
Normalization
Increases CNN's operating speed.
Reshape
Keras works with 3D matrix does not work with 2D so it must be 3D.
Label Encoding
Encode labels to one hot vectors
2 => [0,0,1,0,0,0,0,0,0,0]
****4 => [0,0,0,0,1,0,0,0,0,0]**

In [ ]:
#CNN 
#Define the Model
"""
The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. I choosed to set 32 filters for the two firsts conv2D layers and 64 filters for the two last ones. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

The CNN can isolate features that are useful everywhere from these transformed images (feature maps).

The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

Combining convolutional and pooling layers, CNN are able to combine local features and learn more global features of the image.

Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a propotion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting.
"""

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),padding='Same',activation='relu',input_shape=(28,28,1)))

model.add(Conv2D(filters=64,kernel_size=(5,5),padding='Same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
#model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10,name='Dense10', activation = "softmax"))
print(model.summary())

In [ ]:
##Set the optimiser 
"""

Once our layers are added to the model, we need to set up a score function, a loss function and an optimisation algorithm.

We define the loss function to measure how poorly our model performs on images with known labels. It is the error rate between the oberved labels and the predicted ones. We use a specific form for categorical classifications (>2 classes) called the "categorical_crossentropy".

The most important function is the optimizer. This function will iteratively improve parameters (filters kernel values, weights and bias of neurons ...) in order to minimise the loss.

I choosed RMSprop (with default values), it is a very effective optimizer. The RMSProp update adjusts the Adagrad method in a very simple way in an attempt to reduce its aggressive, monotonically decreasing learning rate. We could also have used Stochastic Gradient Descent ('sgd') optimizer, but it is slower than RMSprop.

The metric function "accuracy" is used is to evaluate the performance our model. This metric function is similar to the loss function, except that the results from the metric evaluation are not used when training the model
"""

In [ ]:
optimizer=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0)


In [ ]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
batch_size = 64
num_classes = 10
epochs = 1
input_shape = (28, 28, 1)

In [ ]:
history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs,validation_data = (X_val, y_val), verbose = 2)

In [ ]:
#Data Augumentation
"""
In order to avoid overfitting problem, we need to expand artificially our handwritten digit dataset. We can make your existing dataset even larger. The idea is to alter the training data with small transformations to reproduce the variations occuring when someone is writing a digit.

For example, the number is not centered The scale is not the same (some who write with big/small numbers) The image is rotated...

Approaches that alter the training data in ways that change the array representation while keeping the label the same are known as data augmentation techniques. Some popular augmentations people use are grayscales, horizontal flips, vertical flips, random crops, color jitters, translations, rotations, and much more.

By applying just a couple of these transformations to our training data, we can easily double or triple the number of training examples and create a very robust model.

"""

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#datagen.fit(X_train)

In [ ]:
"""

For the data augmentation, i choosed to :

Randomly rotate some training images by 10 degrees
Randomly Zoom by 10% some training images
Randomly shift images horizontally by 10% of the width
Randomly shift images vertically by 10% of the height
I did not apply a vertical_flip nor horizontal_flip since it could have lead to misclassify symetrical numbers such as 6 and 9.

Once our model is ready, we fit the training dataset .

"""

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
datagen.fit(X_train)

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] #// batch_size
                              , callbacks=[learning_rate_reduction])
#model.load_weights('best_weights.hdf5')

In [ ]:
#Evaluating the Model

#Training and validation Curves
fig,ax=plt.subplots(2,1)
ax[0].plot(history.history['loss'],color='b',label='Training loss')
ax[0].plot(history.history['val_loss'],color='r',label='validation loss',axes=ax[0])
legend= ax[0].legend(loc='best',shadow='True')
ax[1].plot(history.history['acc'],color='b',label='Training Accuracy')
ax[1].plot(history.history['val_acc'],color='r',label='validation Accuracy')
legend=ax[1].legend(loc='best',shadow='True')

In [ ]:
#confusion matrix
#plot the confusion matrix on validation result to know the drawback of the model


In [ ]:
results=model.predict(X_test)
results=np.argmax(results,axis=1)
results=pd.Series(results,name="Label")

In [ ]:
submission=pd.concat([pd.Series(range(1,28001),name='ImageId'),results],axis=1)
submission.to_csv("cnn_submission.csv",index=False)